In [ ]:
import pandas as pd 
import numpy as np 
vac_data = pd.read_csv('../input/covid-vaccs/country_vaccinations.csv')
vac_data.head()

## Clean Data


In [ ]:
vac_data['iso_code'] = vac_data['iso_code'].fillna('Unknown')
vac_data = vac_data.fillna(0)
vac_data.head()


In [ ]:
vac_data.to_csv('all_data_clean.csv')

### top 20 vaccinated countries  


In [ ]:
top20 = vac_data.groupby('country')['people_vaccinated'].max().reset_index()
top20 = top20.nlargest(20, ['people_vaccinated']).reset_index()
top20.drop(columns='index', inplace=True)


### The US is in front with the most people vaccinated, probably because they had most of the companies that produce these vaccines

In [ ]:
fig = px.bar(top20, x='country', y='people_vaccinated',
             title='Top 20 de paises con la mayor cantidad de la poblacion Vacunada',
            labels={'people_vaccinated':'Personas Vacunadas', 
                    'country':'Paises'})
fig.show()

## top 20 countries per 100 people 

In [ ]:
top_x100 = vac_data.groupby('country')['people_vaccinated_per_hundred'].max().reset_index()
top_x100 = top_x100.nlargest(20, ['people_vaccinated_per_hundred']).reset_index()
top_x100.drop(columns='index', inplace=True)


### I think here we are seeing the countries that are really making a good job getting their population vaccinated. 

In [ ]:
fig = px.bar(top_x100, x='country', y='people_vaccinated_per_hundred',
             title='Top 20 paises con la mayor cantidad de la poblacion Vacunada por cada 100 habitantes',
            labels={'people_vaccinated_per_hundred':'Personas Vacunadas', 
                    'country':'Paises'})
fig.show()

In [ ]:
top_x100.to_csv('top_20x100.csv')

## vaccinated in Paraguay

In [ ]:
py = vac_data.groupby('country')
py = py.get_group('Paraguay')


In [ ]:
py.to_csv('paraguay.csv')

### I'm from Paraguay and the government is not really making a good job getting the vaccines, the have been just sitting there, expecting vaccines from heaven or something

In [ ]:
fig = px.bar(py, x='date', y='people_vaccinated',
             title='Progreso de Vacunacion en Py',
            labels={'people_vaccinated':'Personas Vacunadas', 
                    'date':'fecha'})
fig.show()

In [ ]:
region = vac_data.groupby('country')
country_list=['Paraguay', 'Uruguay', 'Argentina', 'Venezuela','Bolivia','Brazil', 'Chile', 'Colombia', 'Ecuador', 'Peru']
# new_gb = pd.concat( [ region.get_group(c) for country in country_list if country == ])
# new_gb

## get latam countries



In [ ]:
g = vac_data.groupby('country')
argentina = g.get_group('Argentina')

In [ ]:
uruguay = g.get_group('Uruguay')
bolivia = g.get_group('Bolivia')
brasil = g.get_group('Brazil')
chile = g.get_group('Chile')
colombia = g.get_group('Colombia')
ecuador = g.get_group('Ecuador')
peru = g.get_group('Peru')
venezuela = g.get_group('Venezuela')


In [ ]:
country_list = [py, uruguay, argentina, bolivia, brasil, chile, colombia, ecuador, peru, venezuela]
region = pd.concat(country_list).reset_index()
region.drop(columns='index', inplace=True)


In [ ]:
latam_m = region.groupby('country').max().reset_index()
latam_m = latam_m.nlargest(10, ['people_vaccinated']).reset_index()
latam_m.drop(columns='index', inplace=True)

In [ ]:
latam_m.to_csv('Vacunas_latino.csv')

### Brazil is the leading country in South America

In [ ]:
fig = px.bar(latam_m, x='country', y='people_vaccinated',
             title='Poblacion Vacunada en America del Sur, por pais',
            labels={'people_vaccinated':'Personas Vacunadas', 
                    'country':'Paises'})
fig.show()

### It seems like Chile is making an excellent job getting their people vaccinated, we probably should look into what are they doing there.

In [ ]:


fig = px.bar(latam_m, x='country', y='people_vaccinated_per_hundred',
             title='Vacunados por cada 100 habitantes',
            labels={'people_vaccinated_per_hundred':'Personas Vacunadas', 
                    'country':'Paises'})
fig.show()

## add population column

In [ ]:
popu = pd.read_csv('../input/population-by-country-2020/population_by_country_2020.csv')


In [ ]:
top20_countries = list(top20['country'])
pop_country = list(popu['Country (or dependency)'])
pop_popu = list(popu['Population (2020)'])

top20_pop = []
pop_20_pop = []

for i in range(len(top20_countries)):
    for j in range(len(pop_country)):
        if(top20_countries[i]==pop_country[j]):
            top20_pop.append(pop_country[j])
            pop_20_pop.append(pop_popu[j])

In [ ]:
df_top_20pop = pd.DataFrame()
df_top_20pop['Country'] = top20_pop
df_top_20pop['Population'] = pop_20_pop


In [ ]:
top20 = top20[top20.country.apply(lambda x: x not in ['England'])]


In [ ]:
top20merge = top20.rename(columns={'country':'Country'})
concated = pd.merge(df_top_20pop, top20merge)


## get percentages

In [ ]:
concated['population percentage vaccinated'] = 100 * concated['people_vaccinated'] / concated['Population']
concated = concated.nlargest(19, 'population percentage vaccinated')


### Higher percentage of the population vaccinated 

In [ ]:
fig = px.bar(concated, x='Country', y='population percentage vaccinated',
             title='Top 20 de paises con mayor porcentaje de la poblacion vacunada',
            labels={'population percentage vaccinated':'porcentaje', 
                    'Country':'Paises'})
fig.show()

### Get latam population

In [ ]:
percen_latam = latam_m.groupby('country')['people_vaccinated'].max().reset_index()
latam = list(percen_latam['country'])
pop_country = list(popu['Country (or dependency)'])
pop_popu = list(popu['Population (2020)'])

latam_popu = []
pop_la_pop = []

for i in range(len(latam)):
    for j in range(len(pop_country)):
        if(latam[i]==pop_country[j]):
            latam_popu.append(pop_country[j])
            pop_la_pop.append(pop_popu[j])

In [ ]:
df_latam = pd.DataFrame()
df_latam['Country'] = latam_popu 
df_latam['Population'] = pop_la_pop

In [ ]:
percen_latam = percen_latam.rename(columns={'country':'Country'})
percen_latam = pd.merge(df_latam, percen_latam)

percen_latam['population percentage vaccinated'] = 100 * percen_latam['people_vaccinated'] / percen_latam['Population']
percen_latam = percen_latam.nlargest(10, 'population percentage vaccinated')

### Chile again making a really good job

In [ ]:
fig = px.bar(percen_latam, x='Country', y='population percentage vaccinated',
             title='Latino America por porcentaje de la poblacion vacunada',
            labels={'population percentage vaccinated':'Porcentaje', 
                    'Country':'Paises'})
fig.show()

### Daily vaccinations across the world


In [ ]:
df_daily = vac_data[['date', 'daily_vaccinations']].sort_values('date')
df_daily = df_daily.groupby(['date'])['daily_vaccinations'].sum().reset_index()

### We the vaccinations rates are slowing a little bit down. Maybe there is some FUD?

In [ ]:
fig = px.line(df_daily, x="date", y="daily_vaccinations", title='Daily Vaccination across the World')
fig.show()